In [17]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.python.compiler.tensorrt import trt_convert as trt
from tensorflow.python.platform import gfile


In [2]:
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
x = base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x)
x=Dropout(0.5)(x)
preds=Dense(1,activation='sigmoid')(x) #final layer with softmax activation

W0911 10:08:30.312613 140552366577472 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [3]:
model = Model(inputs=base_model.input, outputs=preds)

In [4]:
for layer in model.layers:
  layer.trinable=True


In [5]:
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_data = datagen.flow_from_directory(directory='chest_xray/train', 
                                         target_size=(299, 299), 
                                         color_mode='rgb', 
                                         class_mode='binary', 
                                         batch_size=32, 
                                         shuffle=True)
val_data = datagen.flow_from_directory(directory='chest_xray/test', 
                                         target_size=(299, 299), 
                                         color_mode='rgb', 
                                         class_mode='binary', 
                                         batch_size=32, 
                                         shuffle=True)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [6]:
model.compile(optimizer='Adam', loss="binary_crossentropy", metrics=['accuracy'])

W0911 10:08:49.199019 140552366577472 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [7]:
step_train = train_data.n // train_data.batch_size
step_val = val_data.n // val_data.batch_size

model.fit_generator(generator=train_data, 
                    shuffle=True, 
                    steps_per_epoch=step_train, 
                    epochs=2, 
                    validation_data=val_data, 
                    validation_steps=step_val)

Epoch 1/2
163/163 [==============================] - 148s 907ms/step - loss: 0.1329 - acc: 0.9546 - val_loss: 4.8987 - val_acc: 0.8339
Epoch 2/2
163/163 [==============================] - 110s 675ms/step - loss: 0.0552 - acc: 0.9804 - val_loss: 0.5833 - val_acc: 0.8553


In [8]:
model.save('chest_model.h5')

In [13]:
model.output

<tf.Tensor 'dense_1/Sigmoid:0' shape=(?, 1) dtype=float32>

In [15]:
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K
from tensorflow.python.framework import graph_io
from tensorflow.python.tools import freeze_graph
from tensorflow.core.protobuf import saver_pb2
from tensorflow.python.training import saver as saver_lib

def convert_keras_to_pb(keras_model, out_names, models_dir, model_filename):
    model = load_model(keras_model)
    K.set_learning_phase(0)
    sess = K.get_session()
    saver = saver_lib.Saver(write_version=saver_pb2.SaverDef.V2)
    checkpoint_path = saver.save(sess, 'saved_ckpt', global_step=0, latest_filename='checkpoint_state')
    graph_io.write_graph(sess.graph, '.', 'tmp.pb')
    freeze_graph.freeze_graph('./tmp.pb', 
                              '',
                              False, checkpoint_path, out_names,
                              "save/restore_all", "save/Const:0",
                              models_dir+model_filename, False, "")
convert_keras_to_pb("chest_model.h5", ['dense_1/Sigmoid'], '', 'chest_model.pb')

IndexError: list index out of range

In [2]:
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow.keras import backend as K
# This line must be executed before loading Keras model.
K.set_learning_phase(0)

In [7]:
from tensorflow.keras.models import load_model
model = load_model('chest_model.h5')
print(model.outputs)
# [<tf.Tensor 'dense_2/Softmax:0' shape=(?, 10) dtype=float32>]
print(model.inputs)
# [<tf.Tensor 'conv2d_1_input:0' shape=(?, 28, 28, 1) dtype=float32>]

W0911 10:33:23.140794 140077484074816 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0911 10:33:23.142192 140077484074816 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0911 10:33:23.143296 140077484074816 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtyp

[<tf.Tensor 'dense_1/Sigmoid:0' shape=(?, 1) dtype=float32>]
[<tf.Tensor 'input_1:0' shape=(?, 299, 299, 3) dtype=float32>]


In [8]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph


frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

W0911 10:33:54.683955 140077484074816 deprecation.py:323] From <ipython-input-8-e018fe30fb29>:28: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
W0911 10:33:54.684833 140077484074816 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/graph_util_impl.py:270: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


In [15]:
# Save to ./model/tf_model.pb
tf.train.write_graph(frozen_graph, "", "chest_model.pb", as_text=False)
import os
os.listdir()

['chest_model.h5',
 'Untitled.ipynb',
 'tmp.pb',
 'checkpoint_state',
 '__MACOSX',
 'chest_xray',
 'saved_ckpt-0.index',
 '.ipynb_checkpoints',
 'saved_ckpt-0.data-00000-of-00001',
 'saved_ckpt-0.meta',
 'chest_model.pb']

In [18]:
# convert (optimize) frozen model to TensorRT model
trt_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,# frozen model
    outputs=['dense_1/Sigmoid'],
    max_batch_size=64,# specify your max batch size
    max_workspace_size_bytes=2*(10**9),# specify the max workspace
    precision_mode="FP16") # precision, can be "FP32" (32 floating point precision) or "FP16"

#write the TensorRT model to be used later for inference
with gfile.FastGFile("chest_TensorRT_model.pb", 'wb') as f:
    f.write(trt_graph.SerializeToString())
print("TensorRT model is successfully stored!")

W0911 11:28:17.856310 140077484074816 deprecation.py:323] From <ipython-input-18-0225cca0ef9b>:10: FastGFile.__init__ (from tensorflow.python.platform.gfile) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.gfile.GFile.


TensorRT model is successfully stored!


In [25]:
# import the needed libraries
import tensorflow as tf
import tensorflow.contrib.tensorrt as trt # must import this although we will not use it explicitly
from tensorflow.python.platform import gfile
import numpy as np

img1 = image.load_img(r'chest_xray/val/NORMAL/' + 'NORMAL2-IM-1438-0001.jpeg', target_size=(299, 299))
img_array1 = image.img_to_array(img1)
img_array_expanded_dims1 = np.expand_dims(img_array1, axis=0)
img1 = preprocess_input(img_array_expanded_dims1)

img2 = image.load_img('chest_xray/val/NORMAL/' + 'NORMAL2-IM-1440-0001.jpeg', target_size=(299, 299))
img_array2 = image.img_to_array(img2)
img_array_expanded_dims2 = np.expand_dims(img_array2, axis=0)
img2 = preprocess_input(img_array_expanded_dims2)

input_img = np.concatenate((img1, img2),axis=0)

In [28]:
input_img.shape

(2, 299, 299, 3)

In [32]:
# function to read a ".pb" model 
# (can be used to read frozen model or TensorRT model)
import time
def read_pb_graph(model):
  with gfile.FastGFile(model,'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
  return graph_def

In [33]:
# variable
TENSORRT_MODEL_PATH = 'chest_TensorRT_model.pb'

graph = tf.Graph()
with graph.as_default():
    with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.50))) as sess:
        # read TensorRT model
        trt_graph = read_pb_graph(TENSORRT_MODEL_PATH)

        # obtain the corresponding input-output tensor
        tf.import_graph_def(trt_graph, name='')
        input = sess.graph.get_tensor_by_name('input_1:0')
        output = sess.graph.get_tensor_by_name('dense_1/Sigmoid:0')

        # in this case, it demonstrates to perform inference for 50 times
        total_time = 0; n_time_inference = 50
        out_pred = sess.run(output, feed_dict={input: input_img})
        for i in range(n_time_inference):
            t1 = time.time()
            out_pred = sess.run(output, feed_dict={input: input_img})
            t2 = time.time()
            delta_time = t2 - t1
            total_time += delta_time
            print("needed time in inference-" + str(i) + ": ", delta_time)
        avg_time_tensorRT = total_time / n_time_inference
        print("average inference time: ", avg_time_tensorRT)

needed time in inference-0:  0.010419130325317383
needed time in inference-1:  0.010721445083618164
needed time in inference-2:  0.011287927627563477
needed time in inference-3:  0.01136159896850586
needed time in inference-4:  0.010973453521728516
needed time in inference-5:  0.011221170425415039
needed time in inference-6:  0.010986566543579102
needed time in inference-7:  0.0097198486328125
needed time in inference-8:  0.01002359390258789
needed time in inference-9:  0.010281562805175781
needed time in inference-10:  0.009742021560668945
needed time in inference-11:  0.010100364685058594
needed time in inference-12:  0.010137796401977539
needed time in inference-13:  0.010174989700317383
needed time in inference-14:  0.009631156921386719
needed time in inference-15:  0.010051727294921875
needed time in inference-16:  0.010152578353881836
needed time in inference-17:  0.009802579879760742
needed time in inference-18:  0.009966373443603516
needed time in inference-19:  0.0104024410247

In [34]:
# variable
FROZEN_MODEL_PATH = 'chest_model.pb'

graph = tf.Graph()
with graph.as_default():
    with tf.Session() as sess:
        # read TensorRT model
        frozen_graph = read_pb_graph(FROZEN_MODEL_PATH)

        # obtain the corresponding input-output tensor
        tf.import_graph_def(frozen_graph, name='')
        input = sess.graph.get_tensor_by_name('input_1:0')
        output = sess.graph.get_tensor_by_name('dense_1/Sigmoid:0')

        # in this case, it demonstrates to perform inference for 50 times
        total_time = 0; n_time_inference = 50
        out_pred = sess.run(output, feed_dict={input: input_img})
        for i in range(n_time_inference):
            t1 = time.time()
            out_pred = sess.run(output, feed_dict={input: input_img})
            t2 = time.time()
            delta_time = t2 - t1
            total_time += delta_time
            print("needed time in inference-" + str(i) + ": ", delta_time)
        avg_time_original_model = total_time / n_time_inference
        print("average inference time: ", avg_time_original_model)
        print("TensorRT improvement compared to the original model:", avg_time_original_model/avg_time_tensorRT)

needed time in inference-0:  0.21779942512512207
needed time in inference-1:  0.02843642234802246
needed time in inference-2:  0.030545711517333984
needed time in inference-3:  0.026583433151245117
needed time in inference-4:  0.026366233825683594
needed time in inference-5:  0.023306846618652344
needed time in inference-6:  0.025398969650268555
needed time in inference-7:  0.0233309268951416
needed time in inference-8:  0.02337360382080078
needed time in inference-9:  0.023790597915649414
needed time in inference-10:  0.030689239501953125
needed time in inference-11:  0.022977828979492188
needed time in inference-12:  0.028777360916137695
needed time in inference-13:  0.02548813819885254
needed time in inference-14:  0.02433300018310547
needed time in inference-15:  0.022630929946899414
needed time in inference-16:  0.027002334594726562
needed time in inference-17:  0.024901390075683594
needed time in inference-18:  0.023601055145263672
needed time in inference-19:  0.0260219573974609